In [10]:
import asyncio
import os

import nest_asyncio
from dotenv import load_dotenv

from telegram import Update
from telegram.ext import (
    ApplicationBuilder, CommandHandler, MessageHandler,
    ContextTypes, filters
)

from yandex_cloud_ml_sdk import YCloudML

from sqlalchemy import create_engine, Column, Integer, BigInteger, String, DateTime, Identity
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
import datetime


In [8]:
# загрузка переменных из .env
load_dotenv()

# получаем токен из окружения
TOKEN = os.getenv("BOT_TOKEN")
POSTGRES_CONNECTION_URL = os.getenv("POSTGRES_CONNECTION_URL")
YCloudML_FOLDER_ID = os.getenv("YCloudML_FOLDER_ID")
YCloudML_AUTH_TOKEN = os.getenv("YCloudML_AUTH_TOKEN")


In [3]:
Base = declarative_base()

class Action(Base):
    __tablename__ = 'actions'
    id = Column(Integer, Identity(), primary_key=True)
    user_id = Column(BigInteger, nullable=False)
    created_datetime = Column(DateTime(timezone=True), server_default=func.now())
    action = Column(String(255), nullable=False)

class Message(Base):
    __tablename__ = 'messages'
    id = Column(Integer, Identity(), primary_key=True)
    user_id = Column(BigInteger, nullable=False)
    message = Column(String, nullable=True)
    reply = Column(String, nullable=True)
    created_datetime = Column(DateTime(timezone=True), server_default=func.now())
    score = Column(Integer, nullable=True)

database = create_engine(POSTGRES_CONNECTION_URL, echo=True)

In [4]:
# Создать таблицы
Base.metadata.create_all(database)

2025-04-16 14:37:38,969 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-04-16 14:37:38,972 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 14:37:39,229 INFO sqlalchemy.engine.Engine select current_schema()
2025-04-16 14:37:39,230 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 14:37:39,483 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-04-16 14:37:39,485 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 14:37:39,761 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 14:37:39,772 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [11]:
def generate_llm_request(user_message):
    full_message = f'''Пользователь прислал сообщение. Ответь ему чтонибудь в хардкорном метал стиле, но без оскорблений и непреличия.
    Не нужно никаких вводных.
    Отвечай на русском языке.
    Твоё сообщение должно быть прямым ответом пользователю.
    Вот сообщение пользователя: {user_message}'''
    return [
        {
            "role": "user",
            "text": full_message,
        },
    ]

def get_llm_reply(message):
    global YCloudML_FOLDER_ID
    global YCloudML_AUTH_TOKEN
    sdk = YCloudML(folder_id=YCloudML_FOLDER_ID, auth=YCloudML_AUTH_TOKEN)
    model = sdk.models.completions("yandexgpt-lite", model_version="rc")
    model = model.configure(temperature=0.3)
    llm_request = generate_llm_request(message)
    result = model.run(llm_request)
    llm_response = '\n'.join([r.text for r in result])
    return llm_response


In [6]:
# команда /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        Session = sessionmaker(bind=database)
        session = Session()
        new_action = Action(user_id=update.message.from_user.id, action='start')
        session.add(new_action)
        session.commit()
        print("Action added successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if 'session' in locals():
            session.close()
    await update.message.reply_text("Привет, бро! Ты готов услышать рёв настоящего металла? ")

# эхо-ответ
async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    bot_message = get_llm_reply(update.message.text)
    try:
        user_id = update.message.from_user.id
        Session = sessionmaker(bind=database)
        session = Session()
        new_action = Action(user_id=user_id, action='answer')
        session.add(new_action)
        session.commit()
        print("Action added successfully.")
        new_message = Message(
            user_id=user_id,
            message=update.message.text,
            reply=bot_message,
            score=0
        )
        session.add(new_message)
        session.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if 'session' in locals():
            session.close()
    await update.message.reply_text(bot_message)

# асинхронный запуск
async def main():
    app = ApplicationBuilder().token(TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))
    print("Бот запущен!")
    await app.run_polling()  # без попытки закрыть loop вручную

In [7]:
# проверяем, нужен ли asyncio workaround
# if __name__ == "__main__":
try:
    asyncio.run(main())
except RuntimeError as e:
    if "asyncio" in str(e) or "running event loop" in str(e):
        print("Обнаружен активный event loop — используем nest_asyncio.")
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(main())

Обнаружен активный event loop — используем nest_asyncio.
Бот запущен!
2025-04-16 14:44:05,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 14:44:05,637 INFO sqlalchemy.engine.Engine INSERT INTO actions (user_id, action) VALUES (%(user_id)s, %(action)s) RETURNING actions.id, actions.created_datetime
2025-04-16 14:44:05,639 INFO sqlalchemy.engine.Engine [generated in 0.00230s] {'user_id': 485220305, 'action': 'answer'}
2025-04-16 14:44:05,902 INFO sqlalchemy.engine.Engine COMMIT
Action added successfully.
2025-04-16 14:44:06,044 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 14:44:06,048 INFO sqlalchemy.engine.Engine INSERT INTO messages (user_id, message, reply, score) VALUES (%(user_id)s, %(message)s, %(reply)s, %(score)s) RETURNING messages.id, messages.created_datetime
2025-04-16 14:44:06,050 INFO sqlalchemy.engine.Engine [generated in 0.00243s] {'user_id': 485220305, 'message': 'йо', 'reply': 'Йоу, бро, давай пожёстче, в ритме блэк-метала!', 'score': 0}
2025-

RuntimeError: Cannot close a running event loop